In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision==0.23.0
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 43.6 MB/s eta 0:00:00


In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
mask_predictor = SamPredictor(sam)

In [ ]:
import os

IMAGE_FOLDER = "/content/drive/MyDrive/truchas"
OUTPUT_FOLDER = "/content/drive/MyDrive/truchasmask2"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
import xml.etree.ElementTree as ET
from jupyter_bbox_widget import BBoxWidget
import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64," + encoded

def extract_bounding_boxes(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []

    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        boxes.append([xmin, ymin, xmax, ymax])

    return boxes

In [ ]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget




In [ ]:
import os
import base64
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import supervision as sv

In [ ]:
def calculate_mask_stability(mask):
    # Aplicar un suavizado Gaussiano para reducir el ruido
    blurred_mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # Detectar contornos de la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular el área total del contorno
    area = sum([cv2.contourArea(c) for c in contours])

    # Calcular la cantidad de ruido (pequeñas regiones dispersas)
    noise = np.sum(mask != blurred_mask)

    # Calcular la suavidad del contorno basado en el perímetro/área
    perimeter = sum([cv2.arcLength(c, True) for c in contours])
    smoothness_ratio = perimeter / (area + 1e-6)  # Añadir epsilon para evitar división por cero

    # Combinar métricas para determinar la estabilidad (a menor valor, mejor estabilidad)
    stability_score = smoothness_ratio + noise * 0.001  # Puedes ajustar el peso del ruido
    return stability_score

In [ ]:
import os
import cv2
import numpy as np

# Asegúrate de haber importado los módulos necesarios como mask_predictor, sv, etc.
def calculate_mask_stability(mask):
    mask = mask.astype(np.uint8) * 255
    # Aplicar un suavizado Gaussiano para reducir el ruido
    blurred_mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # Detectar contornos de la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular el área total del contorno
    area = sum([cv2.contourArea(c) for c in contours])

    # Calcular la cantidad de ruido (pequeñas regiones dispersas)
    noise = np.sum(mask != blurred_mask)

    # Calcular la suavidad del contorno basado en el perímetro/área
    perimeter = sum([cv2.arcLength(c, True) for c in contours])
    smoothness_ratio = perimeter / (area + 1e-6)  # Añadir epsilon para evitar división por cero

    # Aumentar el peso del área: ahora le damos más importancia
    area_weight = 1 / (area**2 + 1e-6)  # Aumentar el peso del área con una raíz cuadrada inversa

    # Combinar métricas para determinar la estabilidad
    stability_score = (smoothness_ratio * 0.5 + noise * 0.0001) * area_weight
    return stability_score, area

def choose_most_stable_and_largest_mask(masks):
    # Calcular la estabilidad y área para cada máscara
    stability_scores = []
    areas = []
    for mask in masks:
        stability_score, area = calculate_mask_stability(mask)
        stability_scores.append(stability_score)
        areas.append(area)

    # Elegir la máscara que tenga el mejor equilibrio entre estabilidad y área
    best_mask_index = np.argmin(stability_scores)
    return masks[best_mask_index], best_mask_index  # Devolver la máscara seleccionada y su índice

def export_and_display_selected_mask(masks, output_folder, mask_index, image_name):
    # Validar que el índice esté dentro del rango de máscaras disponibles
    if mask_index < 0 or mask_index >= len(masks):
        print(f"Índice fuera de rango. Hay {len(masks)} máscaras disponibles.")
        return

    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Seleccionar la máscara específica según el índice
    selected_mask = masks[mask_index]

    # Convertir la máscara seleccionada a imagen binaria (0 o 255)
    binary_mask = (selected_mask * 255).astype(np.uint8)

    # Guardar la máscara seleccionada con el nombre de la imagen original
    mask_path = os.path.join(output_folder, f'{image_name}.png')
    cv2.imwrite(mask_path, binary_mask)

    print(f"Máscara {mask_index} ha sido guardada en {output_folder} como {image_name}.png")

# Proceso principal
for filename in os.listdir(IMAGE_FOLDER):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construir la ruta completa de la imagen
        image_path = os.path.join(IMAGE_FOLDER, filename)
        xml_path = os.path.join(IMAGE_FOLDER, os.path.splitext(filename)[0] + '.xml')
        mask_output_path = os.path.join(OUTPUT_FOLDER, f"{os.path.splitext(filename)[0]}_mask.png")

        if os.path.exists(mask_output_path):
            print(f"Máscara ya existe para {filename}, saltando a la siguiente imagen.")
            continue  # Saltar si la máscara ya existe

        if os.path.exists(xml_path):
            # Cargar la imagen en formato BGR
            image_bgr = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
            mask_predictor.set_image(image_rgb)

            # Extraer bounding boxes del archivo XML
            bounding_boxes = extract_bounding_boxes(xml_path)

            candidate_masks = []

            for bbox in bounding_boxes:
                box = np.array([bbox[0], bbox[1], bbox[2], bbox[3]])

                # Predecir máscaras usando el mask predictor
                masks, scores, logits = mask_predictor.predict(box=box, multimask_output=True)

                # Añadir las máscaras candidatas para evaluación
                for mask in masks:
                    candidate_masks.append(mask)

            # Elegir la mejor máscara usando el criterio de estabilidad y área
            if candidate_masks:
                best_mask, best_mask_index = choose_most_stable_and_largest_mask(candidate_masks)

                # Convertir la mejor máscara a binaria y guardar con el nombre original
                image_base_name = os.path.splitext(filename)[0]  # Obtener el nombre base de la imagen
                export_and_display_selected_mask(candidate_masks, OUTPUT_FOLDER, best_mask_index, image_base_name)

print("Proceso completo para todas las imágenes.")



Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 07570736-94a0-4a8d-982c-ff4f815170a3-290mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 06b705c2-07d2-425f-b118-1adf35fe6caa-400mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 043a3baa-aead-4834-b3ce-bc99dcd5e5fc-190mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 0f9239d0-299c-4c42-9374-7df1d37fffe6-430mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 03fd161e-08ed-4f46-af77-47944e31dc74-290mm.png
Máscara 1 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 02506b90-48e3-4f17-86a3-df0fd4609b36-350mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 104381-0mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 100822-350mm.png
Máscara 2 ha sido guardada en /content/drive/MyDrive/truchasmask2 como 102800-230mm.png
Máscara 2 ha sido guardada en 